# Questions

* Feature exploration
* Explainable classifier
* Thumbnail analysis
* NLP
    * Title analysis
    * Tag compositions
    * Description

---

# Setup

In [9]:
cd ../..

C:\Users\eric\Documents\DIGIPEN\PersonalSVN\Fall22SVN\CSP400


In [76]:
# Global
import numpy as np
import pandas as pd

from sklearn.tree import DecisionTreeClassifier

# Local
from utils.yt_accessor import YouTubeAccessor
from utils.categories import YouTubeCategories

In [75]:
pd.set_option('display.max_columns', 100)

##### Load Data

In [5]:
df = pd.read_pickle("https://squeemos.pythonanywhere.com/static/yt_categories.xz")

In [6]:
df

,queryTime,kind,etag,id,snippet.publishedAt,snippet.channelId,snippet.title,snippet.description,snippet.thumbnails.default.url,snippet.thumbnails.default.width,...,recordingDetails.location.longitude,recordingDetails.location.altitude,localizations.vi.title,localizations.vi.description,recordingDetails.recordingDate,localizations.en-GB.title,localizations.en-GB.description,contentDetails.contentRating.ytRating,localizations.en-IN.title,localizations.en-IN.description
0,2022-10-11 07:01:10+00:00,youtube#video,iKhjshxofWhps4FNEQoyXfcfSHc,KeRiH3G7PIA,2022-09-27 19:30:02+00:00,UCcud9msMXGG45gmU1j8t6pA,"""First Time Being Robbed?"" 👤😅",Watch full movie on Amazon Prime: (Out of Sigh...,https://i.ytimg.com/vi/KeRiH3G7PIA/default.jpg,120,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-10-11 07:01:10+00:00,youtube#video,7T_jrtqM40sui-SxuHQeq9rrUeE,UIGNMccGh8w,2022-10-02 18:20:31+00:00,UCQz2x0BlBM9MVaUW_kYPosg,مشهد سينمائي من فيلم | roman j israel esq,#سينمائيونHD #أقتباسات_افلام #أقتباسات_سينمائي...,https://i.ytimg.com/vi/UIGNMccGh8w/default.jpg,120,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-10-11 07:01:10+00:00,youtube#video,p364it_pskLAO7XCjVUi2bO4bQc,pFI3hVpV9S8,2022-10-07 01:47:40+00:00,UCYIEv9W7RmdpvFkHX7IEmyg,THE HAVE IT ALL TOUR STARTS TOMORROW! 🤩 #short...,,https://i.ytimg.com/vi/pFI3hVpV9S8/default.jpg,120,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-10-11 07:01:10+00:00,youtube#video,DE6bSs3zluCFV51eRVRkJtnPldI,DUnQcJz76Ck,2022-10-06 20:23:11+00:00,UCpJN7kiUkDrH11p0GQhLyFw,THE SUPER MARIO BROS: MOVIE Official Trailer (...,First movie trailer for The Super Mario Bros. ...,https://i.ytimg.com/vi/DUnQcJz76Ck/default.jpg,120,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-10-11 07:01:10+00:00,youtube#video,0uUupdOFBiA_p9DF5gQpmdAeiXI,SS7HXxy3_2c,2022-10-09 05:38:34+00:00,UCqFzWxSCi39LnW1JKFR3efg,Try Guys - SNL,A CNN broadcast is interrupted by breaking new...,https://i.ytimg.com/vi/SS7HXxy3_2c/default.jpg,120,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24950,2022-10-13 01:00:50+00:00,youtube#video,9cZgf5EbAr1eq0d2IljgVJZKQ_g,SwNnW8h-uMA,2022-10-02 19:13:45+00:00,UC-SrCCzkGq0wmSAuRs7EBFg,The Sun IS Actually Real.,,https://i.ytimg.com/vi/SwNnW8h-uMA/default.jpg,120,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24951,2022-10-13 01:00:50+00:00,youtube#video,_2zpmtQn-0nYsIQQZrVW5gPD8Ek,1ncUvtK9D0c,2022-10-11 21:33:11+00:00,UCe7KQjGEUloxCv5yB4fSIOA,THE TRY GUYS ARE REALLY UPSET,#thetryguys #tryguys #nedfulmer #spillsesh,https://i.ytimg.com/vi/1ncUvtK9D0c/default.jpg,120,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24952,2022-10-13 01:00:50+00:00,youtube#video,tlaHUobakPbmGn9NuD-WcsDKgs0,-OZx_egKJ38,2022-10-09 22:28:06+00:00,UC6VAIqNQBc7ggiqvsOHOBqw,HE PUT A TARANTULA ON MY FACE?! (Ping-Pong Gam...,I did a CRAZY challenge... DO NOT ATTEMPT YOUR...,https://i.ytimg.com/vi/-OZx_egKJ38/default.jpg,120,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24953,2022-10-13 01:00:50+00:00,youtube#video,X72TCTzDqMKxDN1fD0OHE8OlTtk,4sGiMcU3tkc,2022-10-01 00:02:49+00:00,UCpuT8AlP9P9EgW_pZ0_xInQ,These best friends finally reunited after 5 ye...,,https://i.ytimg.com/vi/4sGiMcU3tkc/default.jpg,120,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
latest = df.drop_duplicates(subset="id", keep="last", ignore_index=True)

##### Load Categories

In [22]:
categories = YouTubeCategories("https://squeemos.pythonanywhere.com/static/video_categories.json")
categories

In [32]:
categories.id_to_title[23]

'Comedy'

---

# Classifier

* Idea
    * Create explainable classifer that predicts video category and explains the discriminating features by which the decision is made.
* Applications
    * Submit video link/id to show category information.
    * Fill in title, description, thumbnail, tags, etc. to show category probability distribution.

In [46]:
clf_df = latest.copy()

##### Drop Features

In [60]:
clf_df_info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 786 entries, 0 to 785
Data columns (total 76 columns):
 #   Column                                    Non-Null Count  Dtype              
---  ------                                    --------------  -----              
 0   queryTime                                 786 non-null    datetime64[ns, UTC]
 1   kind                                      786 non-null    object             
 2   etag                                      786 non-null    object             
 3   id                                        786 non-null    object             
 4   snippet.publishedAt                       786 non-null    datetime64[ns, UTC]
 5   snippet.channelId                         786 non-null    object             
 6   snippet.title                             786 non-null    object             
 7   snippet.description                       786 non-null    object             
 8   snippet.thumbnails.default.url            786 non-null    ob

In [62]:
# For checking features that contain string for removal
fname = "contentDetails"
[col for col in clf_df.columns if fname in col]

['contentDetails.duration',
 'contentDetails.dimension',
 'contentDetails.definition',
 'contentDetails.caption',
 'contentDetails.licensedContent',
 'contentDetails.projection',
 'contentDetails.regionRestriction.blocked',
 'contentDetails.regionRestriction.allowed',
 'contentDetails.contentRating.ytRating']

In [68]:
[col for col in clf_df.columns if not any(d in col for d in drops)]

['queryTime',
 'kind',
 'etag',
 'id',
 'snippet.publishedAt',
 'snippet.channelId',
 'snippet.title',
 'snippet.description',
 'snippet.channelTitle',
 'snippet.categoryId',
 'snippet.liveBroadcastContent',
 'snippet.defaultLanguage',
 'snippet.localized.title',
 'snippet.localized.description',
 'snippet.defaultAudioLanguage',
 'contentDetails.duration',
 'contentDetails.dimension',
 'contentDetails.definition',
 'contentDetails.caption',
 'contentDetails.licensedContent',
 'contentDetails.projection',
 'status.uploadStatus',
 'status.privacyStatus',
 'status.license',
 'status.embeddable',
 'status.publicStatsViewable',
 'status.madeForKids',
 'statistics.viewCount',
 'statistics.likeCount',
 'statistics.favoriteCount',
 'statistics.commentCount',
 'player.embedHtml',
 'topicDetails.topicCategories',
 'snippet.tags',
 'liveStreamingDetails.actualStartTime',
 'liveStreamingDetails.actualEndTime',
 'liveStreamingDetails.scheduledStartTime',
 'recordingDetails.locationDescription',
 'r

In [72]:
# Feature substrings to drop
drops = (
    "localizations", "liveStreamingDetails", "recordingDetails", 
    "regionRestriction", "ytRating", "thumbnails", "defaultLanguage",
)

# Remove all features with names
clf_df = clf_df.loc[:, [col for col in clf_df.columns if not any(d in col for d in drops)]]

clf_df

,queryTime,kind,etag,id,snippet.publishedAt,snippet.channelId,snippet.title,snippet.description,snippet.channelTitle,snippet.categoryId,...,status.embeddable,status.publicStatsViewable,status.madeForKids,statistics.viewCount,statistics.likeCount,statistics.favoriteCount,statistics.commentCount,player.embedHtml,topicDetails.topicCategories,snippet.tags
0,2022-10-11 07:01:10+00:00,youtube#video,eocF5Kc1DFeq27YghDHrZgMOcaQ,0AIGbmHsEp4,2022-10-10 21:15:01+00:00,UCfw6CRDOg5xQsB-2QNC_KUA,ROBLOX MOVIE THEATER,2 da movies\n\nPLAY: https://www.roblox.com/ga...,Sketch,20,...,True,True,False,157853,6489.0,0,873.0,"<iframe width=""480"" height=""270"" src=""//www.yo...",[https://en.wikipedia.org/wiki/Action-adventur...,"[Sk3tch, sketch, sketch roblox, roblox, roblox..."
1,2022-10-11 07:01:10+00:00,youtube#video,cc9xeWmpr9R8mvlPSf0U7YHlCCY,Arst6Qex4ag,2022-10-10 18:30:18+00:00,UCwCj-mTphfnjBcRag6MEM2A,EASY DRAWING vs IMPOSSIBLE DRAWING,Easy Drawing VS Impossible Drawing In Just Dra...,ProjectJamesify,20,...,True,False,False,113028,2075.0,0,98.0,"<iframe width=""480"" height=""270"" src=""//www.yo...",[https://en.wikipedia.org/wiki/Video_game_cult...,"[projectjamesify, jamesify, just draw, just dr..."
2,2022-10-11 07:01:10+00:00,youtube#video,SIbWANyvX6oLtZOQRI-SKf16Od4,iXyWwu-MBLY,2022-10-07 23:32:39+00:00,UC6ZFN9Tx6xh-skXCuRHCDpQ,"PBS NewsHour full episode, Oct. 7, 2022","Friday on the NewsHour, the latest jobs report...",PBS NewsHour,25,...,True,True,False,192631,1286.0,0,NaN,"<iframe width=""480"" height=""270"" src=""//www.yo...","[https://en.wikipedia.org/wiki/Society, https:...","[PBS NewsHour, NewsHour episode, NewsHour epis..."
3,2022-10-11 07:01:10+00:00,youtube#video,zZuLHGDZ2WA96mAuoOHCed2ev3Y,Sb2680L1dNo,2022-10-06 20:00:25+00:00,UCvixJtaXuNdMPUGdOPcY8Ag,"Trump Visibly Disoriented, Brain Glitches Badl...",💪 Athletic Greens is offering FREE year-supply...,David Pakman Show,25,...,True,True,False,560113,15912.0,0,5607.0,"<iframe width=""480"" height=""270"" src=""//www.yo...",[https://en.wikipedia.org/wiki/Society],"[trump speaks at hispanic event, trump speaks ..."
4,2022-10-11 07:01:10+00:00,youtube#video,z_dgvp7d9jJ08ir7uVXwjAiIijk,sA3LQ9oYlTA,2022-09-30 22:41:04+00:00,UCOrE6g6E4HwdUWaZeITOAaQ,$500 NYC apartment #shorts #nyc,,CALEB SIMPSON,22,...,True,True,False,9666965,618049.0,0,11779.0,"<iframe width=""480"" height=""270"" src=""//www.yo...",[https://en.wikipedia.org/wiki/Lifestyle_(soci...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
781,2022-10-13 01:00:50+00:00,youtube#video,9cZgf5EbAr1eq0d2IljgVJZKQ_g,SwNnW8h-uMA,2022-10-02 19:13:45+00:00,UC-SrCCzkGq0wmSAuRs7EBFg,The Sun IS Actually Real.,,Miniminuteman,27,...,True,True,False,1643838,216321.0,0,2792.0,"<iframe width=""480"" height=""270"" src=""//www.yo...",[https://en.wikipedia.org/wiki/Lifestyle_(soci...,NaN
782,2022-10-13 01:00:50+00:00,youtube#video,_2zpmtQn-0nYsIQQZrVW5gPD8Ek,1ncUvtK9D0c,2022-10-11 21:33:11+00:00,UCe7KQjGEUloxCv5yB4fSIOA,THE TRY GUYS ARE REALLY UPSET,#thetryguys #tryguys #nedfulmer #spillsesh,Spill Sesh,22,...,True,True,False,703243,25228.0,0,1961.0,"<iframe width=""480"" height=""270"" src=""//www.yo...",[https://en.wikipedia.org/wiki/Lifestyle_(soci...,"[the try guys, try guys, try guys ned, the try..."
783,2022-10-13 01:00:50+00:00,youtube#video,tlaHUobakPbmGn9NuD-WcsDKgs0,-OZx_egKJ38,2022-10-09 22:28:06+00:00,UC6VAIqNQBc7ggiqvsOHOBqw,HE PUT A TARANTULA ON MY FACE?! (Ping-Pong Gam...,I did a CRAZY challenge... DO NOT ATTEMPT YOUR...,NichLmao,22,...,True,True,False,1606490,NaN,0,13142.0,"<iframe width=""480"" height=""270"" src=""//www.yo...",[https://en.wikipedia.org/wiki/Lifestyle_(soci...,NaN
784,2022-10-13 01:00:50+00:00,youtube#video,X72TCTzDqMKxDN1fD0OHE8OlTtk,4sGiMcU3tkc,2022-10-01 00:02:49+00:00,UCpuT8AlP9P9EgW_pZ0_xInQ,These best friends finally reunited after 5 ye...,,Dylan Anderson,17,...,True,True,False,3069224,270450.0,0,1384.0,"<iframe width=""480"" height=""270"" src=""//www.yo...",[ht

In [73]:
clf_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 786 entries, 0 to 785
Data columns (total 33 columns):
 #   Column                          Non-Null Count  Dtype              
---  ------                          --------------  -----              
 0   queryTime                       786 non-null    datetime64[ns, UTC]
 1   kind                            786 non-null    object             
 2   etag                            786 non-null    object             
 3   id                              786 non-null    object             
 4   snippet.publishedAt             786 non-null    datetime64[ns, UTC]
 5   snippet.channelId               786 non-null    object             
 6   snippet.title                   786 non-null    object             
 7   snippet.description             786 non-null    object             
 8   snippet.channelTitle            786 non-null    object             
 9   snippet.categoryId              786 non-null    int64              
 10  snippet.liveBr

In [77]:
clf_df

,queryTime,kind,etag,id,snippet.publishedAt,snippet.channelId,snippet.title,snippet.description,snippet.channelTitle,snippet.categoryId,snippet.liveBroadcastContent,snippet.localized.title,snippet.localized.description,snippet.defaultAudioLanguage,contentDetails.duration,contentDetails.dimension,contentDetails.definition,contentDetails.caption,contentDetails.licensedContent,contentDetails.projection,status.uploadStatus,status.privacyStatus,status.license,status.embeddable,status.publicStatsViewable,status.madeForKids,statistics.viewCount,statistics.likeCount,statistics.favoriteCount,statistics.commentCount,player.embedHtml,topicDetails.topicCategories,snippet.tags
0,2022-10-11 07:01:10+00:00,youtube#video,eocF5Kc1DFeq27YghDHrZgMOcaQ,0AIGbmHsEp4,2022-10-10 21:15:01+00:00,UCfw6CRDOg5xQsB-2QNC_KUA,ROBLOX MOVIE THEATER,2 da movies\n\nPLAY: https://www.roblox.com/ga...,Sketch,20,none,ROBLOX MOVIE THEATER,2 da movies\n\nPLAY: https://www.roblox.com/ga...,en,PT8M4S,2d,hd,True,True,rectangular,processed,public,youtube,True,True,False,157853,6489.0,0,873.0,"<iframe width=""480"" height=""270"" src=""//www.yo...",[https://en.wikipedia.org/wiki/Action-adventur...,"[Sk3tch, sketch, sketch roblox, roblox, roblox..."
1,2022-10-11 07:01:10+00:00,youtube#video,cc9xeWmpr9R8mvlPSf0U7YHlCCY,Arst6Qex4ag,2022-10-10 18:30:18+00:00,UCwCj-mTphfnjBcRag6MEM2A,EASY DRAWING vs IMPOSSIBLE DRAWING,Easy Drawing VS Impossible Drawing In Just Dra...,ProjectJamesify,20,none,EASY DRAWING vs IMPOSSIBLE DRAWING,Easy Drawing VS Impossible Drawing In Just Dra...,en,PT11M40S,2d,hd,True,True,rectangular,processed,public,youtube,True,False,False,113028,2075.0,0,98.0,"<iframe width=""480"" height=""270"" src=""//www.yo...",[https://en.wikipedia.org/wiki/Video_game_cult...,"[projectjamesify, jamesify, just draw, just dr..."
2,2022-10-11 07:01:10+00:00,youtube#video,SIbWANyvX6oLtZOQRI-SKf16Od4,iXyWwu-MBLY,2022-10-07 23:32:39+00:00,UC6ZFN9Tx6xh-skXCuRHCDpQ,"PBS NewsHour full episode, Oct. 7, 2022","Friday on the NewsHour, the latest jobs report...",PBS NewsHour,25,none,"PBS NewsHour full episode, Oct. 7, 2022","Friday on the NewsHour, the latest jobs report...",en-US,PT56M46S,2d,hd,True,True,rectangular,processed,public,youtube,True,True,False,192631,1286.0,0,NaN,"<iframe width=""480"" height=""270"" src=""//www.yo...","[https://en.wikipedia.org/wiki/Society, https:...","[PBS NewsHour, NewsHour episode, NewsHour epis..."
3,2022-10-11 07:01:10+00:00,youtube#video,zZuLHGDZ2WA96mAuoOHCed2ev3Y,Sb2680L1dNo,2022-10-06 20:00:25+00:00,UCvixJtaXuNdMPUGdOPcY8Ag,"Trump Visibly Disoriented, Brain Glitches Badl...",💪 Athletic Greens is offering FREE year-supply...,David Pakman Show,25,none,"Trump Visibly Disoriented, Brain Glitches Badl...",💪 Athletic Greens is offering FREE year-supply...,en,PT7M38S,2d,hd,True,True,rectangular,processed,public,youtube,True,True,False,560113,15912.0,0,5607.0,"<iframe width=""480"" height=""270"" src=""//www.yo...",[https://en.wikipedia.org/wiki/Society],"[trump speaks at hispanic event, trump speaks ..."
4,2022-10-11 07:01:10+00:00,youtube#video,z_dgvp7d9jJ08ir7uVXwjAiIijk,sA3LQ9oYlTA,2022-09-30 22:41:04+00:00,UCOrE6g6E4HwdUWaZeITOAaQ,$500 NYC apartment #shorts #nyc,,CALEB SIMPSON,22,none,$500 NYC apartment #shorts #nyc,,NaN,PT49S,2d,hd,True,False,rectangular,processed,public,youtube,True,True,False,9666965,618049.0,0,11779.0,"<iframe width=""480"" height=""270"" src=""//www.yo...",[https://en.wikipedia.org/wiki/Lifestyle_(soci...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
781,2022-10-13 01:00:50+00:00,youtube#video,9cZgf5EbAr1eq0d2IljgVJZKQ_g,SwNnW8h-uMA,2022-10-02 19:13:45+00:00,UC-SrCCzkGq0wmSAuRs7EBFg,The Sun IS Actually Real.,,Miniminuteman,27,none,The Sun IS Actually Real.,,NaN,PT1M,2d,hd,True,False,rectangular,processed,public,youtube,True,True,False,1643838,216321.0,0,2792.0,"<iframe width=""480"" height=""270"" src=""//www.yo...",[https://en.wikip